In [ ]:
# DV9: Display histograms of average pollutant concentrations per province, including global averages
#  ## DV-9 Average concentration – histogram
# The interface allow the user to choose a pollutant and a time window and 
# visualize in a histogram the average for each province (along with the global average)
# concentration of the selected pollutant over a selected time period.
# [Optional: same average but for each municipality

# - button select pollutant
# - button select start day
# - button select end day
# - button select start mounth
# - button select end mounth
# - button select start year
# - button select end year
# - button select administrative level

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Output dinamici
output1 = widgets.Output()
output2 = widgets.Output()

# Variabili globali
start_date = None
end_date = None

# Province della Lombardia
province_lombardia = [
    "Bergamo", "Brescia", "Como", "Cremona", "Lecco", "Lodi",
    "Mantova", "Milano", "Monza e della Brianza", "Pavia",
    "Sondrio", "Varese"
]

# Dropdown province (attivato solo dopo che le date sono definite)
dropdown_prov = widgets.Dropdown(
    options=province_lombardia,
    description='Provincia:'
)

# Dropdown inquinanti
dropdown_pollutant = widgets.Dropdown(
    options=[
        "Ammoniaca", "Arsenico", "Benzo(a)pirene", "Benzene", "Biossido di Azoto",
        "Biossido di Zolfo", "BlackCarbon", "Cadmio", "Monossido di Azoto",
        "Monossido di Carbonio", "Nikel", "Ossidi di Azoto", "Ozono", "PM10",
        "PM10 (SM2005)", "Particelle sospese PM2.5", "Particolato Totale Sospeso",
        "Piombo"
    ],
    description='Inquinante:'
)

# Callback dopo selezione inquinante
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output1:
            clear_output()
            global start_date, end_date
            start_date = widgets.DatePicker(description='Inizio:')
            end_date = widgets.DatePicker(description='Fine:')

            def check_dates(_):
                if start_date.value and end_date.value:
                    with output2:
                        clear_output()
                        display(dropdown_prov)

            start_date.observe(check_dates, names='value')
            end_date.observe(check_dates, names='value')

            display(widgets.VBox([start_date, end_date]))

# Collega evento
dropdown_pollutant.observe(on_dropdown_change, names='value')

# Visualizzazione
display(dropdown_pollutant)
display(output1)
display(output2)


Dropdown(description='Inquinante:', options=('Ammoniaca', 'Arsenico', 'Benzo(a)pirene', 'Benzene', 'Biossido d…

Output()

Output()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

df = pd.read_csv("air_quality_data.csv")

# Assumiamo colonne:
# 'pollutant', 'value', 'province', 'municipality', 'date'
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [ ]:
pollutants = df['pollutant'].unique()
years = sorted(df['year'].unique())
months = sorted(df['month'].unique())
days = sorted(df['day'].unique())
levels = ['Province', 'Municipality']

dropdown_pollutant = widgets.Dropdown(options=pollutants, description='Pollutant:')
dropdown_year_start = widgets.Dropdown(options=years, description='Start Year:')
dropdown_year_end = widgets.Dropdown(options=years, description='End Year:')
dropdown_month_start = widgets.Dropdown(options=months, description='Start Month:')
dropdown_month_end = widgets.Dropdown(options=months, description='End Month:')
dropdown_day_start = widgets.Dropdown(options=days, description='Start Day:')
dropdown_day_end = widgets.Dropdown(options=days, description='End Day:')
dropdown_level = widgets.Dropdown(options=levels, description='Level:')
button = widgets.Button(description="Visualize")


In [ ]:
def on_button_clicked(b):
    clear_output(wait=True)
    display(dropdown_pollutant, dropdown_year_start, dropdown_year_end,
            dropdown_month_start, dropdown_month_end,
            dropdown_day_start, dropdown_day_end,
            dropdown_level, button)

    # Filtro dei dati
    pollutant = dropdown_pollutant.value
    start_date = pd.Timestamp(dropdown_year_start.value, dropdown_month_start.value, dropdown_day_start.value)
    end_date = pd.Timestamp(dropdown_year_end.value, dropdown_month_end.value, dropdown_day_end.value)
    level = 'province' if dropdown_level.value == 'Province' else 'municipality'

    df_filtered = df[(df['pollutant'] == pollutant) & 
                     (df['date'] >= start_date) & 
                     (df['date'] <= end_date)]

    # Media per livello selezionato
    df_avg = df_filtered.groupby(level)['value'].mean().sort_values(ascending=False)
    global_avg = df_filtered['value'].mean()

    # Plot
    plt.figure(figsize=(12, 6))
    df_avg.plot(kind='bar')
    plt.axhline(global_avg, color='red', linestyle='--', label=f'Global Avg: {global_avg:.2f}')
    plt.title(f"{pollutant} average concentration ({level.title()})")
    plt.ylabel("Avg concentration")
    plt.xlabel(level.title())
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    

In [ ]:
button.on_click(on_button_clicked)

display(dropdown_pollutant, dropdown_year_start, dropdown_year_end,
        dropdown_month_start, dropdown_month_end,
        dropdown_day_start, dropdown_day_end,
        dropdown_level, button)